# ItemList

[TableOfContents]

Let's first create an `ItemList` instance, which we use in the rest of this section.

In [1]:
from tohu.tohu_items_class import make_tohu_items_class
from tohu.item_list import ItemList

In [2]:
Quux = make_tohu_items_class("Quux", field_names=["aa", "bb", "cc", "dd"])

In [3]:
items = [
    Quux(aa=30, bb='672EF2', cc='Johnny', dd=False),
    Quux(aa=32, bb='250204', cc='David', dd=True),
    Quux(aa=55, bb='679DAE', cc='Angela', dd=False),
    Quux(aa=43, bb='91554C', cc='Pamela', dd=True),
    Quux(aa=56, bb='8EA713', cc='Blake', dd=True),
]

In [4]:
item_list = ItemList(items, Quux)
item_list

<ItemList containing 5 items>

## Exporting an `ItemList` to various formats

### `to_df()`

When calling `.to_df()` without any arguments, all fields from all items will be exported.

In [5]:
item_list.to_df()

,aa,bb,cc,dd
0,30,672EF2,Johnny,False
1,32,250204,David,True
2,55,679DAE,Angela,False
3,43,91554C,Pamela,True
4,56,8EA713,Blake,True


### `head()`

There is also a `.head()` method, which is analogous to the one for pandas dataframes.

In [6]:
item_list.head(3)

,aa,bb,cc,dd
0,30,672EF2,Johnny,False
1,32,250204,David,True
2,55,679DAE,Angela,False


## Specifying which columns to export, and in which order

We can export only a subset of columns, and/or rearrange their order, by specifying a list of field names. In this example we export only the three columns `cc`, `aa`, `dd` (in this order) instead of the full set of columns `aa`, `bb`, `cc`, `dd`.

In [7]:
item_list.to_df(fields=["cc", "aa", "dd"])

,cc,aa,dd
0,Johnny,30,False
1,David,32,True
2,Angela,55,False
3,Pamela,43,True
4,Blake,56,True


## Custom field names

If we want to rename the columns we can pass a dictionary of the form `{<new_column_name>: <existing_field_name>}`, for example:

In [8]:
item_list.to_df(fields={"First Name": "cc", "Age": "aa"})

,First Name,Age
0,Johnny,30
1,David,32
2,Angela,55
3,Pamela,43
4,Blake,56


## Accessing nested field values

Let's create a second `ItemList` instance which contains nested items.

In [9]:
Foo = make_tohu_items_class("Foo", field_names=["xx", "yy"])
Bar = make_tohu_items_class("Bar", field_names=["rr", "ss"])

In [10]:
items_2 = [
    Quux(aa=30, bb=Foo(xx='672EF2', yy=Bar(rr=153, ss="Engineer")), cc='Johnny', dd=False),
    Quux(aa=32, bb=Foo(xx='250204', yy=Bar(rr=193, ss="Therapist")), cc='David', dd=True),
    Quux(aa=55, bb=Foo(xx='679DAE', yy=Bar(rr=101, ss="Author")), cc='Angela', dd=False),
    Quux(aa=43, bb=Foo(xx='91554C', yy=Bar(rr=138, ss="Scientist")), cc='Pamela', dd=True),
    Quux(aa=56, bb=Foo(xx='8EA713', yy=Bar(rr=147, ss="Consultant")), cc='Blake', dd=True),
]

item_list_2 = ItemList(items_2, Quux)

If we simply call `to_df()` without specifying the `fields` argument, the cells in column `bb` will contain the full nested values of the items of type `Foo`.

In [11]:
item_list_2.to_df()

,aa,bb,cc,dd
0,30,"Foo(xx='672EF2', yy=Bar(rr=153, ss='Engineer'))",Johnny,False
1,32,"Foo(xx='250204', yy=Bar(rr=193, ss='Therapist'))",David,True
2,55,"Foo(xx='679DAE', yy=Bar(rr=101, ss='Author'))",Angela,False
3,43,"Foo(xx='91554C', yy=Bar(rr=138, ss='Scientist'))",Pamela,True
4,56,"Foo(xx='8EA713', yy=Bar(rr=147, ss='Consultant'))",Blake,True


If we are only interested in some of the individual values in the nested items `Foo` or `Bar`, it is possible to "reach into" them and extract those values by using the usual `.` notation for accessing attributes.

In [12]:
fields = {"Name": "cc", "ID": "bb.xx", "Job description": "bb.yy.ss"}

item_list_2.to_df(fields=fields)

,Name,ID,Job description
0,Johnny,672EF2,Engineer
1,David,250204,Therapist
2,Angela,679DAE,Author
3,Pamela,91554C,Scientist
4,Blake,8EA713,Consultant


Let's verify that an error is raised if an attribute name specified in the `fields` argument doesn't exist at any level.

In [13]:
import pytest

with pytest.raises(AttributeError, match="Quux' object has no attribute 'bbb'"):
    item_list_2.to_df(fields={"Job description": "bbb.yy.ss"})

with pytest.raises(AttributeError, match="Foo' object has no attribute 'yyy'"):
    item_list_2.to_df(fields={"Job description": "bb.yyy.ss"})

with pytest.raises(AttributeError, match="Bar' object has no attribute 'sss'"):
    item_list_2.to_df(fields={"Job description": "bb.yy.sss"})